<a href="https://colab.research.google.com/github/FelicianoRoblesAndrade/Python/blob/main/ICDM_modelado_y_evaluacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Universidad de Sonora

Introducción a la Ciencia de Datos y sus Metodologías

Adrian.vazquez@unison.mx

Ejercicio. Modelado y evaluación

Esta actividad consiste en practicar el modelado y la evaluación del modelo para un problema de ciencia de datos. Para ello, se utilizan los datos de la calidad del agua superficial del varios años (dicha
información puede obtenerse de la página de CONAGUA).

1. Con base en lo discutido sobre la contaminación del agua, elaborar una descripción del problema.

2. Elaborar un gráfico de serie temporal para indicar la contaminación del agua a nivel nacional comparado con la contaminación en el estado de Sonora (estos cálculos deben realizarse basados
en las mediciones y no en los sitios).

3. Elaborar una gráfica de barras para indicar, por año, la cantidad de contaminación que ha habido a nivel nacional a través de los años. Cada barra debe representar la cantidad de agua contaminada y la cantidad de agua limpia para cada año (estos cálculos deben realizarse basados en las mediciones realizadas indistintamente del lugar de medición).

4. Suponer que se tienen N sitios de monitoreo, de los cuales M son considerados un conjunto de prueba. Después de aplicar un algoritmo a varios modelos se obtuvieron las siguientes matrices
de confusión:

Modelo 1

| VP=1153  | VN=891 |
|----------|--------|
| FP=27    | FN=693 |

Modelo 2

| VP=1246  | VN=871 |
|----------|--------|
| FP=47    | FN=600 |

Modelo 3

| VP=1538  | VN=745 |
|----------|--------|
| FP=173   | FN=308 |

Modelo 4

| VP=1746  | VN=571 |
|----------|--------|
| FP=347    | FN=100 |

5. Con base a los modelos anteriores, calcular para cada caso:


a. Sensibilidad

b. Precisión

c. Exactitud

d. Especificidad


6. Con base a los cálculos de propiedades de los modelos, ¿Cuál modelo consideras que representa
mejores resultados para el problema de la contaminación del agua? Justificar tu respuesta.


## 1. Con base en lo discutido sobre la contaminación del agua, elaborar una descripción del problema.



El problema es conseguir la contaminacion del agua en base a factores

## 2. Elaborar un gráfico de serie temporal para indicar la contaminación del agua a nivel nacional comparado con la contaminación en el estado de Sonora (estos cálculos deben realizarse basados en las mediciones y no en los sitios).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install pyxlsb

In [55]:
columnas+tox

['CLAVE SITIO',
 'TIPO CUERPO DE AGUA',
 'FECHA REALIZACIÓN',
 'COLI_FEC',
 'E_COLI',
 'DBO_TOT',
 'DQO_TOT',
 'OD_%',
 'ENTEROC_FEC',
 'SST',
 'TOX_D_48_UT',
 'TOX_D_48_SUP_UT',
 'TOX_D_48_FON_UT',
 'TOX_FIS_SUP_15_UT',
 'TOX_FIS_FON_15_UT',
 'TOX_V_15_UT']

In [117]:
#url='https://files.conagua.gob.mx/aguasnacionales/TODOS%20LOS%20MONITOREOS.xlsb' # Por si se quiere todo #
columnas = ['TIPO CUERPO DE AGUA','FECHA REALIZACIÓN',
            'COLI_FEC','E_COLI','DBO_TOT','DQO_TOT','OD_%','ENTEROC_FEC','SST',
            ]
tox=['TOX_D_48_UT','TOX_D_48_SUP_UT','TOX_D_48_FON_UT','TOX_FIS_SUP_15_UT',
     'TOX_FIS_FON_15_UT','TOX_V_15_UT',
     ]
url='https://files.conagua.gob.mx/aguasnacionales/RESULTADOS-SONORA.xlsb'
df_Son = pd.read_excel(url, engine='pyxlsb', sheet_name=1, usecols=columnas+tox) #usecols="A,D,E,G:QJ" 'CLAVE SITIO',

In [118]:
df_Son.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5949 entries, 0 to 5948
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   TIPO CUERPO DE AGUA  5949 non-null   object
 1   FECHA REALIZACIÓN    5949 non-null   int64 
 2   COLI_FEC             5768 non-null   object
 3   E_COLI               2945 non-null   object
 4   DBO_TOT              2932 non-null   object
 5   DQO_TOT              2930 non-null   object
 6   TOX_D_48_UT          2565 non-null   object
 7   TOX_D_48_SUP_UT      364 non-null    object
 8   TOX_D_48_FON_UT      275 non-null    object
 9   TOX_FIS_SUP_15_UT    2017 non-null   object
 10  TOX_FIS_FON_15_UT    1445 non-null   object
 11  TOX_V_15_UT          2600 non-null   object
 12  OD_%                 2645 non-null   object
 13  ENTEROC_FEC          1647 non-null   object
 14  SST                  4721 non-null   object
dtypes: int64(1), object(14)
memory usage: 697.3+ KB


In [119]:
df=df_Son[~df_Son['TIPO CUERPO DE AGUA'].isin(['SUBTERRÁNEO'])] # Quitamos los elementos subterraneos
df=df.reset_index(drop=True) # Reseteamos el indice

In [120]:
df2=df.copy()
df2['FECHA REALIZACIÓN']=pd.to_datetime(df['FECHA REALIZACIÓN'],origin='1899-12-30',unit='D') # Se convierte a datatime
df2['tox'] = df2[tox].apply(lambda row: 1 if row.notna().any() else float("nan"), axis=1) # Si cualquier registro de toxicidad es distinto de 0, tomar la toxicidad como 1
df2.tail()

,TIPO CUERPO DE AGUA,FECHA REALIZACIÓN,COLI_FEC,E_COLI,DBO_TOT,DQO_TOT,TOX_D_48_UT,TOX_D_48_SUP_UT,TOX_D_48_FON_UT,TOX_FIS_SUP_15_UT,TOX_FIS_FON_15_UT,TOX_V_15_UT,OD_%,ENTEROC_FEC,SST,tox
4667,COSTERO (HUMEDAL),2018-02-20,6131,NaN,NaN,NaN,NaN,NaN,NaN,<1,<1,NaN,NaN,51,77.1429,1.0
4668,COSTERO (HUMEDAL),2018-04-24,175,NaN,NaN,NaN,NaN,NaN,NaN,<1,<1,NaN,NaN,<10,85.8333,1.0
4669,COSTERO (HUMEDAL),2018-06-24,175,NaN,NaN,NaN,NaN,NaN,NaN,<1,<1,NaN,NaN,<10,33,1.0
4670,COSTERO (HUMEDAL),2018-11-05,1483,NaN,NaN,NaN,NaN,NaN,NaN,<1,NaN,NaN,NaN,528,331.4286,1.0
4671,COSTERO (HUMEDAL),2020-10-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<1,NaN,NaN,NaN,2909,76.5714,1.0


In [188]:
df3=df2.drop(columns=['TIPO CUERPO DE AGUA']+tox)
df3.tail()

,FECHA REALIZACIÓN,COLI_FEC,E_COLI,DBO_TOT,DQO_TOT,OD_%,ENTEROC_FEC,SST,tox
4667,2018-02-20,6131,NaN,NaN,NaN,NaN,51,77.1429,1.0
4668,2018-04-24,175,NaN,NaN,NaN,NaN,<10,85.8333,1.0
4669,2018-06-24,175,NaN,NaN,NaN,NaN,<10,33,1.0
4670,2018-11-05,1483,NaN,NaN,NaN,NaN,528,331.4286,1.0
4671,2020-10-05,NaN,NaN,NaN,NaN,NaN,2909,76.5714,1.0


In [189]:
amarillo=['E_COLI','COLI_FEC','SST','OD_%']
rojo=['DBO_TOT','DQO_TOT','tox','ENTEROC_FEC']

In [192]:
# 1=Rojo si los factores rojos no son NAN y rojo no es 1
df3['rojo'] = df3[rojo].notna().any(axis=1).astype(int)

# 1=Amarillo si los factores amarillos no son NAN y rojo no es 1
df3['amarillo'] = df3[amarillo].notna().any(axis=1) & (df3['rojo'] != 1)
df3['amarillo'] = df3['amarillo'].astype(int)

# 1=Verde si todas las columnas de interes son NAN
df3['verde'] = df3[amarillo+rojo].isna().all(axis=1).astype(int)

df3.tail()

,FECHA REALIZACIÓN,COLI_FEC,E_COLI,DBO_TOT,DQO_TOT,OD_%,ENTEROC_FEC,SST,tox,rojo,amarillo,verde
4667,2018-02-20,6131,NaN,NaN,NaN,NaN,51,77.1429,1.0,1,0,0
4668,2018-04-24,175,NaN,NaN,NaN,NaN,<10,85.8333,1.0,1,0,0
4669,2018-06-24,175,NaN,NaN,NaN,NaN,<10,33,1.0,1,0,0
4670,2018-11-05,1483,NaN,NaN,NaN,NaN,528,331.4286,1.0,1,0,0
4671,2020-10-05,NaN,NaN,NaN,NaN,NaN,2909,76.5714,1.0,1,0,0


In [195]:
df4=df3.drop(columns=amarillo+rojo)
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4672 entries, 0 to 4671
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   FECHA REALIZACIÓN  4672 non-null   datetime64[ns]
 1   rojo               4672 non-null   int64         
 2   amarillo           4672 non-null   int64         
 3   verde              4672 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 146.1 KB
